In [1]:
import optuna
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import re

In [2]:
def load_study(study_name):
    journal_name = f'{study_name}.log'
    study_storage = optuna.storages.JournalStorage(
        optuna.storages.JournalFileStorage(journal_name)) 
    for study in study_storage.get_all_studies():
        study_name = study_storage.get_study_name_from_id(study._study_id)
        break
    print('Loading study', study_name)
    study = optuna.study.load_study(study_name=study_name, storage=study_storage)
    return study

In [3]:
def parse_studies(prefix='criteo_', suffix='.log', pattern='*.log'):
    def helper():
        for filename in glob('*.log'):
            study_name = filename.removesuffix(suffix)
            clean_name = study_name.removeprefix(prefix)
            embed_dim_match = re.search('_([0-9]+)$', clean_name)
            embed_dim = int(embed_dim_match.group(1))
            exp_name = clean_name.removesuffix(embed_dim_match.group(0))
            yield exp_name, embed_dim, study_name
    return list(helper())

In [4]:
parse_studies()

[('splines_0', 16, 'criteo_splines_0_16'),
 ('bins', 18, 'criteo_bins_18'),
 ('splines_3', 16, 'criteo_splines_3_16'),
 ('bins', 14, 'criteo_bins_14'),
 ('splines_3', 14, 'criteo_splines_3_14'),
 ('splines_0', 14, 'criteo_splines_0_14'),
 ('bins', 12, 'criteo_bins_12'),
 ('splines_3', 12, 'criteo_splines_3_12'),
 ('splines_0', 12, 'criteo_splines_0_12'),
 ('bins', 16, 'criteo_bins_16'),
 ('splines_3', 18, 'criteo_splines_3_18'),
 ('splines_0', 18, 'criteo_splines_0_18'),
 ('bins', 20, 'criteo_bins_20'),
 ('splines_3', 20, 'criteo_splines_3_20'),
 ('splines_0', 20, 'criteo_splines_0_20'),
 ('bins', 22, 'criteo_bins_22'),
 ('splines_3', 22, 'criteo_splines_3_22'),
 ('splines_0', 22, 'criteo_splines_0_22'),
 ('bins', 24, 'criteo_bins_24'),
 ('splines_3', 24, 'criteo_splines_3_24'),
 ('splines_0', 24, 'criteo_splines_0_24'),
 ('bins', 40, 'criteo_bins_40'),
 ('splines_3', 40, 'criteo_splines_3_40'),
 ('splines_0', 40, 'criteo_splines_0_40'),
 ('bins', 10, 'criteo_bins_10'),
 ('splines_3', 

In [5]:
trials_df = []
for experiment, embed_dim, study_name in parse_studies():
    study = load_study(study_name)
    print(f'Study {study_name} has {len(study.trials)} trials')

    trial_data = [trial.params | 
                  trial.user_attrs | 
                  dict(val_loss=trial.value, study_name=study_name, experiment=experiment, embed_dim=embed_dim)
                  for trial in [study.best_trial]]
    trial_data_df = pd.DataFrame.from_records(trial_data)
    trials_df.append(trial_data_df)

trials_df = pd.concat(trials_df, axis=0)
trials_df

Loading study criteo_splines_0
Study criteo_splines_0_16 has 51 trials
Loading study criteo_bins_18
Study criteo_bins_18 has 50 trials
Loading study criteo_splines_3
Study criteo_splines_3_16 has 51 trials


/tmp/ipykernel_1207161/2670384594.py:3: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  study_storage = optuna.storages.JournalStorage(


Loading study criteo_bins_14
Study criteo_bins_14 has 100 trials
Loading study criteo_splines_3_14
Study criteo_splines_3_14 has 96 trials
Loading study criteo_splines_0_14
Study criteo_splines_0_14 has 100 trials
Loading study criteo_bins_12
Study criteo_bins_12 has 50 trials
Loading study criteo_splines_3_12
Study criteo_splines_3_12 has 45 trials
Loading study criteo_splines_0_12
Study criteo_splines_0_12 has 49 trials
Loading study criteo_bins
Study criteo_bins_16 has 50 trials
Loading study criteo_splines_3_18
Study criteo_splines_3_18 has 50 trials
Loading study criteo_splines_0_18
Study criteo_splines_0_18 has 50 trials
Loading study criteo_bins_20
Study criteo_bins_20 has 50 trials
Loading study criteo_splines_3_20
Study criteo_splines_3_20 has 50 trials
Loading study criteo_splines_0_20
Study criteo_splines_0_20 has 50 trials
Loading study criteo_bins_22
Study criteo_bins_22 has 50 trials
Loading study criteo_splines_3_22
Study criteo_splines_3_22 has 50 trials
Loading study c

,lr,l2reg,random_seed,degree,best_epoch,test_loss,val_loss,study_name,experiment,embed_dim
0,0.000165,3.923746e-05,42,0,1,0.444775,0.444593,criteo_splines_0_16,splines_0,16
0,0.000150,1.679050e-05,42,0,1,0.445195,0.445027,criteo_bins_18,bins,18
0,0.000169,4.634381e-05,42,3,1,0.444511,0.444401,criteo_splines_3_16,splines_3,16
0,0.000187,5.451417e-05,42,0,1,0.445053,0.444966,criteo_bins_14,bins,14
0,0.000168,7.962894e-04,42,3,1,0.444947,0.444813,criteo_splines_3_14,splines_3,14
0,0.000177,5.660776e-08,42,0,1,0.445127,0.444916,criteo_splines_0_14,splines_0,14
0,0.000200,2.494279e-04,42,0,1,0.445159,0.444966,criteo_bins_12,bins,12
0,0.000200,2.494279e-04,42,3,1,0.445042,0.444854,criteo_splines_3_12,splines_3,12
0,0.000185,3.852165e-05,42,0,1,0.444993,0.444820,criteo_splines_0_12,splines_0,12
0,0.000169,4.634381e-05,42,0,1,0.445006,0.444897,criteo_bins_16,bins,16


In [6]:
trials_df[['experiment', 'embed_dim', 'lr', 'l2reg']].pivot(columns=['experiment'], index='embed_dim', values=['lr', 'l2reg'])

lr                         l2reg                            
experiment      bins splines_0 splines_3      bins     splines_0     splines_3
embed_dim                                                                     
10          0.000185  0.000185  0.000159  0.000039  3.852165e-05  3.470267e-05
12          0.000200  0.000185  0.000200  0.000249  3.852165e-05  2.494279e-04
14          0.000187  0.000177  0.000168  0.000055  5.660776e-08  7.962894e-04
16          0.000169  0.000165  0.000169  0.000046  3.923746e-05  4.634381e-05
18          0.000150  0.000153  0.000153  0.000017  4.011123e-05  4.011123e-05
20          0.000158  0.000150  0.000143  0.000089  1.679050e-05  1.001807e-08
22          0.000142  0.000147  0.000146  0.000041  6.358423e-06  4.634381e-05
24          0.000135  0.000124  0.000130  0.000058  7.973484e-05  8.988797e-04
40          0.000291  0.000291  0.000462  0.000010  9.846739e-06  1.152645e-07

In [7]:
pivoted = trials_df[['experiment', 'embed_dim', 'val_loss']].pivot(columns=['experiment'], index='embed_dim', values=['val_loss'])
pivoted

val_loss                    
experiment      bins splines_0 splines_3
embed_dim                               
10          0.445396  0.445543  0.445437
12          0.444966  0.444820  0.444854
14          0.444966  0.444916  0.444813
16          0.444897  0.444593  0.444401
18          0.445027  0.444538  0.444770
20          0.444594  0.444592  0.444383
22          0.444485  0.444645  0.444691
24          0.444591  0.444747  0.444597
40          0.445151  0.445237  0.445153

In [8]:
pivoted = trials_df[['experiment', 'embed_dim', 'test_loss']].pivot(columns=['experiment'], index='embed_dim', values=['test_loss'])
pivoted

test_loss                    
experiment      bins splines_0 splines_3
embed_dim                               
10          0.445568  0.445738  0.445571
12          0.445159  0.444993  0.445042
14          0.445053  0.445127  0.444947
16          0.445006  0.444775  0.444511
18          0.445195  0.444679  0.444897
20          0.444743  0.444707  0.444566
22          0.444677  0.444759  0.444863
24          0.444737  0.444818  0.444815
40          0.445240  0.445340  0.445217

In [9]:
lift_pct = (100 * (1 - pivoted.iloc[:, 1:] / pivoted.iloc[:, 0].values[:, np.newaxis]))
lift_pct

test_loss          
experiment splines_0 splines_3
embed_dim                     
10         -0.038294 -0.000800
12          0.037080  0.026201
14         -0.016780  0.023756
16          0.052041  0.111220
18          0.115976  0.067090
20          0.008036  0.039690
22         -0.018397 -0.041715
24         -0.018198 -0.017544
40         -0.022497  0.005022